In [7]:
import os
import shutil
import random
import glob

In [28]:
keyword = 'adele'

In [29]:
word_path = '/shareddrive/working/data_code/data/neg_data/spcmd/all_words/original/*'
environment_path = '/shareddrive/working/data_code/data/neg_data/envornment/chunks/1s_chunks/*'
noise_path = '/shareddrive/working/data_code/data/neg_data/_background_noise_/chunks/1s_chunks/*'
recording_path = '/shareddrive/working/data_code/data/neg_data/internet_recordings/chunks/1s_chunks/*'
aug_recording_path = '/shareddrive/working/data_code/data/neg_data/internet_recordings/chunks/1s_chunks_aug/*'

In [30]:
keyword_path = f'/shareddrive/working/data_code/data/{keyword}/augmented/*'
keyword_folders = glob.glob(keyword_path)
keyword_files = [j for i in keyword_folders for j in glob.glob(f'{i}/*')]

In [31]:
unknown_files = list()
avg_files = int(len(keyword_files)/8)

In [32]:
word_files = glob.glob(word_path)
environment_files = glob.glob(environment_path)
noise_files = glob.glob(noise_path)
recording_files = glob.glob(recording_path)
aug_recording_files = glob.glob(aug_recording_path)
len(environment_files),len(noise_files),len(word_files),len(recording_files),len(aug_recording_files)

(1109, 660, 64721, 7511, 14483)

In [33]:
unknown_files.extend(random.sample(aug_recording_files,int(avg_files*1.8)))
unknown_files.extend(recording_files)
# file_path_and_labels.extend([(i,0) for i in aug_recording_files])
unknown_files.extend(random.sample(word_files,avg_files))
unknown_files.extend(noise_files)
unknown_files.extend(environment_files)
random.shuffle(unknown_files)

In [34]:
len(keyword_files),len(unknown_files),unknown_files[:3],keyword_files[:3]

(14250,
 14266,
 ['/shareddrive/working/data_code/data/neg_data/spcmd/all_words/original/three_1543.wav',
  '/shareddrive/working/data_code/data/neg_data/envornment/chunks/1s_chunks/env_668.wav',
  '/shareddrive/working/data_code/data/neg_data/internet_recordings/chunks/1s_chunks/967f0bbc017315.wav'],
 ['/shareddrive/working/data_code/data/adele/augmented/band_pass/bp_Adele_0.wav',
  '/shareddrive/working/data_code/data/adele/augmented/band_pass/bp_Adele_1.wav',
  '/shareddrive/working/data_code/data/adele/augmented/band_pass/bp_Adele_10.wav'])

In [35]:
base_dir = '/shareddrive/working/keyword_spotting/data'

In [36]:
test_split = 0.1
validation_split = 0.1
training_split = 1 - (test_split + validation_split)

# Folder names
main_folders = ['training', 'testing', 'validation']
subfolders = [keyword, 'unknown']


In [37]:
for folder in main_folders:
    for subfolder in subfolders:
        os.makedirs(os.path.join(base_dir, folder, subfolder), exist_ok=True)


In [38]:
def distribute_files(files, folder_name):
    random.shuffle(files)  # Shuffle to ensure randomness

    # Determine number of files per split
    test_count = int(len(files) * test_split)
    validation_count = int(len(files) * validation_split)
    training_count = len(files) - test_count - validation_count

    # Split files into training, testing, and validation
    test_files = files[:test_count]
    validation_files = files[test_count:test_count + validation_count]
    training_files = files[test_count + validation_count:]

    # Move files to respective folders in the base directory
    for file in test_files:
        shutil.copy2(file, os.path.join(base_dir, 'testing', folder_name, os.path.basename(file)))
    for file in validation_files:
        shutil.copy2(file, os.path.join(base_dir, 'validation', folder_name, os.path.basename(file)))
    for file in training_files:
        shutil.copy2(file, os.path.join(base_dir, 'training', folder_name, os.path.basename(file)))

In [39]:
distribute_files(keyword_files, keyword)
distribute_files(unknown_files, 'unknown')
